# 1) Data Collection - US Hit Streaming Songs Project
By Jeremias Figueroa

Ever since 10 years old, I was fascinated by music charts such as the Billboard Hot 100. Like many Gen-Z, I grew up with streaming services, and I became fascinated by how music changed as a result of the on-demand streaming era. As a result of this passion, I collected and continue to collect week-to-week numbers since 2017 on the weekly on-demand audio streams top hit songs recieve in the United States. Specifically, I collect data from the HITS Daily Double magazine on this Google Sheet: https://docs.google.com/spreadsheets/d/165OdLYjLt4AgeqP5S5PunRonDkpp28nueHLFv994bPk/edit?usp=sharing. 

This project aims to analyze the characteristics of hit songs and what styles of music does the general public prefer to listen to. It utilizes various Python packages such as gspread to connect Python with Google Sheets and spotipy which hosts Spotify's API data.

__The following Notebook is the first part of four-part series. This one, specifically, collects data from Google Sheets and updates the Google Sheet's data utilizing Google and Spotify's API__. The quick summary of the steps taken are as followed:
- Importing packages
- Adding Spotify Credentials
- Importing the Google Sheet with streams per week
- Searching Uniform Resource Indicators (URIs) from Spotify using Google Sheet info
- Manually inputting songs with no URI found from the previous step
- Obtaining Spotify's API song information
- Joining all data together
- Saving data to Google Sheets for easier access

# Imports

First, install the following (using "pip install _insert_package_here_").
These packages are to be used in the following notebooks as well.
- spotipy
- pandas
- gspread
- oauth2client
- df2gspread
- matplotlib
- seaborn
- sklearn

Next, we import the following for this notebook:

In [1]:
#to access spotipy
import configparser
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

#to allow interaction with underlying operating system
import os

#to parse through spotipy's output
import json

#for data analysis
import numpy as np
import pandas as pd

#to remove the limits on a dataframe's columns displayed
pd.set_option('display.max_columns', None)

#to access google sheets with python
import gspread
from google.oauth2 import service_account
from oauth2client.service_account import ServiceAccountCredentials

#to edit google sheets from python
from df2gspread import df2gspread as d2g

#to ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Spotify API Client Credentials
First, ensure that you have a Spotify For Developers account. Then, using https://developer.spotify.com/dashboard/applications, make an new app and get client ID and client secret ID. 

It is recommendable practice to hide passwords from code. Therefore, environmental variables are to be created in the following:

##  CONVERT THE FOLLOWING TO CODE (W/ YOUR OWN SPOTIFY CREDENTIALS):

- The following creates environmental variables:

%env SPOTIPY_CLIENT_ID=YOUR_SPOTIFY_APP_CLIENT_ID

%env SPOTIPY_CLIENT_SECRET=YOUR_SPOTIFY_APP_CLIENT_SECRET_ID

%env SPOTIPY_REDIRECT_URI=http://127.0.0.1:8080/
        
- my code and its output is hidden to protect privacy

In [3]:
#import spotify client credentials 
spotify_client_id = os.environ['SPOTIPY_CLIENT_ID']
spotify_secret = os.environ['SPOTIPY_CLIENT_SECRET']
spotify_redirect_uri = os.environ['SPOTIPY_REDIRECT_URI']

#create a Spotify instance
client_credentials_manager = SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

#to confirm 
print(sp)

# Importing the Google Sheet

Use song information from the following Google Sheet: https://docs.google.com/spreadsheets/d/165OdLYjLt4AgeqP5S5PunRonDkpp28nueHLFv994bPk/edit?usp=sharing

This Google Sheet is a collection of hit songs from HITS Daily Double, a music industry trade publication, and their weekly Overall Song Streams chart. The data includes how many times a song was streamed week-to-week for most weeks since 2017 (with the exception of holiday weeks, weeks when holiday music regularly push out non-holiday songs out of the top spots of music charts). 

In order to be able to use this data on your own device, create a copy of the Google Sheet to your own Drive. Also, follow the instructions in the following link to connect Google Drive with Python: https://github.com/PrettyPrinted/youtube_video_code/tree/master/2021/10/14/How%20to%20Use%20Google%20Sheets%20With%20Python%20(2021)

For this python notebook, the file for our service account is the json file with the Google API keys downloaded from Google Developers Console. Please reference the link for more instructions.

In [4]:
#service account
sa = gspread.service_account(filename="gspread_service_account.json")

#sheet
sh = sa.open("HITS Streaming Songs")

#worksheet
wks = sh.worksheet("HITS Streaming Songs")

#uses all values in the worksheet for the data frame
gsheets_df = pd.DataFrame(wks.get_all_values())

header = gsheets_df.iloc[0] #isolate first row as header
gsheets_df = gsheets_df[1:] #get rid of header in original df
gsheets_df.columns = header

#set 'index' as index
gsheets_df = gsheets_df.set_index('index')

gsheets_df['title'] = gsheets_df['title'].str.rstrip(' ')
gsheets_df['artist'] = gsheets_df['artist'].str.rstrip(' ')
gsheets_df['album'] = gsheets_df['album'].str.rstrip(' ')

#convert index from string to int
gsheets_df.index = gsheets_df.index.astype(int)

#fill in blank values with 0
gsheets_df = gsheets_df.replace(r'^\s*$', "0", regex=True)

#convert all numbers-as-strings to numbers from row 4 in python's index and onwards
for col_name in gsheets_df.columns.to_list()[4:len(gsheets_df.columns.to_list())-1]:
    
    #get rid of strings' commas and convert strings to integers
    gsheets_df[col_name] = gsheets_df[col_name].str.replace(',','').astype('int')
    
#turn peak_date strings to date format
gsheets_df['peak_date'] = pd.to_datetime(gsheets_df['peak_date'])

gsheets_df

,title,artist,album,peak_date,streams_2017_to_present,total_streams_2020s,peak_week_streams,current_year,2022-09-16,2022-09-09,2022-09-02,2022-08-26,2022-08-19,2022-08-12,2022-08-05,2022-07-29,2022-07-22,2022-07-15,2022-07-08,2022-07-01,2022-06-24,2022-06-17,2022-06-10,2022-06-03,2022-05-27,2022-05-20,2022-05-13,2022-05-06,2022-04-29,2022-04-22,2022-04-15,2022-04-08,2022-04-01,2022-03-25,2022-03-18,2022-03-11,2022-03-04,2022-02-25,2022-02-18,2022-02-11,2022-02-04,2022-01-28,2022-01-21,2022-01-14,2022-01-07,2021-12-31,2021-12-24,2021-12-17,2021-12-10,2021-12-03,2021-11-26,2021-11-19,2021-11-12,2021-11-05,2021-10-29,2021-10-22,2021-10-15,2021-10-08,2021-10-01,2021-09-24,2021-09-17,2021-09-10,2021-09-03,2021-08-27,2021-08-20,2021-08-13,2021-08-06,2021-07-30,2021-07-23,2021-07-16,2021-07-09,2021-07-02,2021-06-25,2021-06-18,2021-06-11,2021-06-04,2021-05-28,2021-05-21,2021-05-14,2021-05-07,2021-04-30,2021-04-23,2021-04-16,2021-04-09,2021-04-02,2021-03-26,2021-03-19,2021-03-12,2021-03-05,2021-02-26,2021-02-19,2021-02-12,2021-02-05,2021-01-29,2021-01-22,2021-01-15,2021-01-08,2021-01-01,2020-12-25,2020-12-18,2020-12-11,2020-12-04,2020-11-27,2020-11-20,2020-11-13,2020-11-06,2020-10-30,2020-10-23,2020-10-16,2020-10-09,2020-10-02,2020-09-25,2020-09-18,2020-09-11,2020-09-04,2020-08-28,2020-08-21,2020-08-14,2020-08-07,2020-07-31,2020-07-24,2020-07-17,2020-07-10,2020-07-03,2020-06-26,2020-06-19,2020-06-12,2020-06-05,2020-05-29,2020-05-22,2020-05-15,2020-05-08,2020-05-01,2020-04-24,2020-04-17,2020-04-10,2020-04-03,2020-03-27,2020-03-20,2020-03-13,2020-03-06,2020-02-28,2020-02-21,2020-02-14,2020-02-07,2020-01-31,2020-01-24,2020-01-17,2020-01-10,2020-01-03,2019-12-27,2019-12-20,2019-12-13,2019-12-06,2019-11-29,2019-11-22,2019-11-15,2019-11-08,2019-11-01,2019-10-25,2019-10-18,2019-10-11,2019-10-04,2019-09-27,2019-09-20,2019-09-13,2019-09-06,2019-08-30,2019-08-23,2019-08-16,2019-08-09,2019-08-02,2019-07-26,2019-07-19,2019-07-12,2019-07-05,2019-06-28,2019-06-21,2019-06-14,2019-06-07,2019-05-31,2019-05-24,2019-05-17,2019-05-10,2019-05-03,2019-04-26,2019-04-19,2019-04-12,2019-04-05,2019-03-29,2019-03-22,2019-03-15,2019-03-08,2019-03-01,2019-02-22,2019-02-15,2019-02-08,2019-02-01,2019-01-25,2019-01-18,2019-01-11,2019-01-04,2018-12-28,2018-12-21,2018-12-14,2018-12-07,2018-11-30,2018-11-23,2018-11-16,2018-11-09,2018-11-02,2018-10-26,2018-10-19,2018-10-12,2018-10-05,2018-09-28,2018-09-21,2018-09-14,2018-09-07,2018-08-31,2018-08-24,2018-08-17,2018-08-10,2018-08-03,2018-07-27,2018-07-20,2018-07-13,2018-07-06,2018-06-29,2018-06-22,2018-06-15,2018-06-08,2018-06-01,2018-05-25,2018-05-18,2018-05-11,2018-05-04,2018-04-27,2018-04-20,2018-04-13,2018-04-06,2018-03-30,2018-03-23,2018-03-16,2018-03-09,2018-03-02,2018-02-23,2018-02-16,2018-02-09,2018-02-02,2018-01-26,2018-01-19,2018-01-12,2018-01-05,2017-12-29,2017-12-15,2017-12-08,2017-12-01,2017-11-24,2017-11-17,2017-11-10,2017-11-03,2017-10-27,2017-10-20,2017-10-13,2017-10-06,2017-09-29,2017-09-22,2017-09-15,2017-09-08,2017-09-01,2017-08-25,2017-08-18,2017-08-11,2017-08-04,2017-07-28,2017-07-21,2017-07-14,2017-07-07,2017-06-30,2017-06-23,2017-06-16,2017-06-09,2017-06-02,2017-05-26,2017-05-19,2017-05-12,2017-05-05,2017-04-28,2017-04-21,2017-04-14,2017-04-07,2017-03-31,total_2022,total_2021,total_2020,total_2019,total_2018,total_2017,units,holiday_music,remix_version,playlist_index
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Closer,The Chainsmokers,Closer,2016-09-09,10948414,0,5528407,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

# Get song URIs from Spotipy

A URI (Uniform Resource Indicator) is a unique link to a song/album/playlist on Spotify. The following takes the song titles, album titles, and artists names from the Google Sheets and uses them as keywords to search for each song's unique URI. 

However, when searching up a song on Spotify, numerous versions of the same song can be returned (single version vs album version, explicit version vs clean version, clean song from a clean parent album or clean song from an explicit parent album). For this analysis (and for the creation of a playlist on another python document!), songs are taken from the explicit version of their parent album/single/EP, regardless of whether the song itself is explicit or clean. If no explicit version of the parent album/single/EP exists, the original clean version is to be used.

The URIs (and explicit status) for each song in the Google Sheet are to be retrieved using the following functions:

In [5]:
def search_for_URI(data):
    """
    Returns track URIs and the explicit status for a song

        Parameters:
            data (series): series containing a song title, artist, and the song's album
        
        Returns:
            song_info_list (list): list of song title, artist, album title, song URI and explicit status
                #otherwise, returns just the song title, artist, and album title
    """
    song_to_search = data['title'].replace("’","").replace("'","").lower()
    album_to_search = data['album'].replace("’","").replace("'","").lower()
    to_search = song_to_search + " " + data['artist']
    
    #this function searches for q on Spotify
    result = sp.search(q=to_search)
        
    #loops through all results from search
    for song_info_index in range(len(result['tracks']['items'])): 

        #checks for results' album to match the input data's album title (taking into account apostrophe formats)
        if result['tracks']['items'][song_info_index]['album']['name'].replace("’","").replace("'","").lower() == album_to_search:
                        
            #checks for results' song title to match input data's song title (taking into account apostrophe formats)
            if result['tracks']['items'][song_info_index]['name'].replace("’","").replace("'","").lower() == song_to_search:     
                
                #empty list to be filled with name/artist/album/explicit/uri info
                song_info_list = []
                
                #checks if a song is explicit       
                if result['tracks']['items'][song_info_index]['explicit'] == True:
                    song_info_list.append(result['tracks']['items'][song_info_index]['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['artists'][0]['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['album']['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['explicit'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['uri'])
                    return song_info_list
                
                #first, if a song is NOT explicit
                if result['tracks']['items'][song_info_index]['explicit'] == False:
                    song_info_list.append(result['tracks']['items'][song_info_index]['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['artists'][0]['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['album']['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['explicit'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['uri'])
                    return song_info_list
    
    #return just the song title/artist/album title if the URI is not found
    return [data['title'],data['artist'],data['album'],"",""]

In [6]:
#create a list of song info: song title, artist, album, explicit or not, track URI
uri_list = []
[uri_list.append(search_for_URI(gsheets_df.loc[song_info_index+1,'title':'album'])) for song_info_index in range(len(gsheets_df.loc[:]))]

#convert list to DataFrame
uri_df = pd.DataFrame(uri_list,columns=["song","artist","album","explicit","uri"])

#change index to start at 1
uri_df.index = range(1,len(uri_df)+1)
uri_df


,song,artist,album,explicit,uri
1,Closer,The Chainsmokers,Closer,False,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw
2,Fake Love,Drake,More Life,True,spotify:track:343YBumqHu19cGoGARUTsd
3,Starboy,The Weeknd,Starboy,True,spotify:track:7MXVkk9YMctZqd1Srtv4MB
4,Reminder,The Weeknd,Starboy,True,spotify:track:37F0uwRSrdzkBiuj0D5UHI
5,Both (feat. Drake),Gucci Mane,The Return of East Atlanta Santa,True,spotify:track:5tFep7dXGd7vEJ668wTPux
...,...,...,...,...,...
2239,Put It On Me,YoungBoy Never Broke Again,Realer 2,True,spotify:track:2FWeNvUo79OeiHWoR1bUsO
2240,Shut Down,BLACKPINK,BORN PINK,False,spotify:track:0ARKW62l9uWIDYMZTUmJHF
2241,Static,Steve Lacy,Gemini Rights,True,spotify:track:4OmfWzukSVD140NiAIEjem
2242,STAR WALKIN' (League of Legends Worlds Anthem),Lil Nas X,STAR WALKIN' (League of Legends Worlds Anthem),False,spotify:track:0bs5Nbnk6nDjxcgqfehx9v


## Check for songs that don't have a URI

In [7]:
#create a dataframe with all songs that have no URI
data_to_clean = uri_df[uri_df['uri'] == ""]
data_to_clean.reset_index(drop=True, inplace=True)
data_to_clean

,song,artist,album,explicit,uri
0,Someone You Loved,Lewis Capaldi,Divinely Uninspired To A Hellish Extent (Exten...,,
1,Costa Rica,Dreamville,Revenge Of The Dreamers III,,
2,Levitating,Dua Lipa,Future Nostalgia (The Moonlight Edition),,


# Searching for missing URIs

Sometimes, not all URIs are found. This can be for various reasons (i.e. a song title changes).

Use the following functions to both search for missing URIs and input them into the dataframe

In [8]:
def song_uri_search(q):
    """
    Prints out different URIs (and other info) from a resulting search on spotipy
    
    Parameter:
        q (str): a string to look up using spotipy's search function
    """
    result = sp.search(q) 
    
    for song_info_dict in range(len(result['tracks']['items'])):
        print("Song: ", result['tracks']['items'][song_info_dict]['name'])
        print("Artist: ", result['tracks']['items'][song_info_dict]['artists'][0]['name'])
        print("Album: ", result['tracks']['items'][song_info_dict]['album']['name'])
        print("Explicit status: ", result['tracks']['items'][song_info_dict]['explicit'])
        print("URI: ", result['tracks']['items'][song_info_dict]['uri'],"\n")

In [9]:
def data_cleaning_func(data_to_clean, uri_df):
    #loop through every song in data_to_clean
    for i in range(0,len(data_to_clean)):
        print("Please follow the following steps to fill in the missing URI(s)." + "\n")
        song_to_search = data_to_clean.iloc[i]['song'].replace("’","").replace("'","")
        album_to_search = data_to_clean.iloc[i]['album'].replace("’","").replace("'","")
        artist_to_search = data_to_clean.iloc[i]['artist']
            
        #concatenate the song title, artist name, and album title to search
        to_search = song_to_search + " " + artist_to_search + " " + album_to_search

        result = sp.search(to_search)
        print("The following results are the search results from searching: '" + to_search + "'")
        print("\n")
            
        #call on song_uri_search function to print out search results
        song_uri_search(to_search)
        print("Search the URI(s) on your browser to double check the song is correct")
            
        #ensure user obtains desired result from the above search
        print("Did you obtain a desired result? Please type 'yes' or 'no': ")
        desired_result = str(input())
            
        #ensure desired_result is a valid input
        while (desired_result != 'yes')  and (desired_result != 'no'):
            print("Please give a valid input. Type either 'yes' or 'no': ")
            desired_result = str(input())
            
        #run the following in case the desired output is not available in the search above
        while desired_result == 'no':
            print("\nPlease manually search the information for this song (be as specific as possible): ")
            manual_search = str(input())
            song_uri_search(manual_search)
                
            #ask if another search is necessary
            print("\nWould you like to make another search? Please type in 'yes' or 'no': ")
            another_search = str(input())
                
            #ensure request for another search is valid
            while (another_search != 'yes')  and (another_search != 'no'):
                print("\nPlease give a valid input. Type either 'yes' or 'no': ")
                another_search = str(input())
                
            if another_search == "no":
                desired_result = 'yes'
            if another_search == "yes":
                desired_result = 'no'
            
        #the index of the song being searched, obtained from uri_df using song and album titles from data_to_clean
        missing_song_index = uri_df[(uri_df['song']==data_to_clean.iloc[i]['song']) & (uri_df['album']==data_to_clean.iloc[i]['album'])].index.values[0]
            
        #obtain explicit status of the song. Have user input the status because numerous songs can result from the search
        print("\nPlease copy and paste the explicit status of your selected song.")
        print("Alternatively, type in 'True' or 'False' (make sure the first character is capitalized): ")
        explicit_status = str(input())
            
        #check if explicit_status is valid
        while (explicit_status != "True")  and (explicit_status != "False"):
            print("\nPlease give a valid input. Type either 'True' or 'False': ")
            explicit_status = str(input())
            
        #to convert the string to a boolean
        explicit_status = eval(explicit_status)
            
        #obtain URI of the song. Have user input the URI because numerous songs can result from the search
        print("\nPlease copy and paste the entire URI for your selected song here: ")
        uri = str(input())
            
        #to give the user a chance to re-enter the URI in case they mistyped it
        print("\nDouble check that your URI is correct. Ready to confirm? Please enter 'yes' or 'no': ")
        confirmation = str(input())
            
        #check if confirmation is valid
        while (confirmation != 'yes')  and (confirmation != 'no'):
            print("\nPlease give a valid input. Type either 'yes' or 'no': ")
            confirmation = str(input())
            
        #ask for user to input URI again if they do not confirm they want to move foward. Continue asking after every input for uri
        if confirmation == 'no':
            print("\nPlease retype your URI here: ")
            uri = str(input())
                
            print("\nDouble check that your URI is correct. Ready to confirm? Please enter 'yes' or 'no': ")
            confirmation = str(input())
                
        #edit the uri dataframe!
        uri_df.loc[missing_song_index]['explicit'] = explicit_status
        uri_df.loc[missing_song_index]['uri'] = uri
            
        print("\nThe URI/explicit status for " + data_to_clean.loc[i]['song'] + " by " + artist_to_search + " is now updated on the uri_df!")
            
    #create a dataframe with all songs that have no URI
    data_to_clean = uri_df[uri_df['uri'] == ""]
    data_to_clean.reset_index(drop=True, inplace=True)
    
    print("\nAll URI/explicit statuses are accounted for!")

In [10]:
if len(data_to_clean) > 0:
    data_cleaning_func(data_to_clean, uri_df)

Please follow the following steps to fill in the missing URI(s).

The following results are the search results from searching: 'Someone You Loved Lewis Capaldi Divinely Uninspired To A Hellish Extent (Extended Edition)'


Song:  Someone You Loved
Artist:  Lewis Capaldi
Album:  Divinely Uninspired To A Hellish Extent (Extended Edition)
Explicit status:  False
URI:  spotify:track:0vGWBo1b6KyE9klUQfxqwJ 

Song:  Someone You Loved
Artist:  Lewis Capaldi
Album:  Divinely Uninspired To A Hellish Extent
Explicit status:  False
URI:  spotify:track:7qEHsqek33rTcFNT9PFqLf 

Song:  Someone You Loved - Live From The BRIT Awards, London 2020
Artist:  Lewis Capaldi
Album:  To Tell The Truth I Can't Believe We Got This Far EP
Explicit status:  False
URI:  spotify:track:1ecJjQT2Y6c6m5n14vt7u9 

Song:  Someone You Loved - Live from Capitol Studios / 1 Mic 1 Take
Artist:  Lewis Capaldi
Album:  Someone You Loved (Live from Capitol Studios / 1 Mic 1 Take)
Explicit status:  False
URI:  spotify:track:5Y3GMn

In [11]:
#Because Drake's "What's Next" is the only one that returned a clean version when it wasn't supposed to (due to problems with the apostrophes), perform the following
uri_df.loc[1543]['explicit'] = True
uri_df.loc[1543]['uri'] = "spotify:track:3aQem4jVGdhtg116TmJnHz"

In [12]:
print("Success! The entire dataframe is now updated with song URIs! Please wait for Spotify's track features to be retreived.")

Success! The entire dataframe is now updated with song URIs! Please wait for Spotify's track features to be retreived.


## Get spotipy's API Information

Spotify has a plethora of information on a song's characteristics. The following functions/cells create a dataframe with song characteristic information on each song from the Google Sheet. 

__Information of what each variable means can be found here__: https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features

In [13]:
def getTrackFeatures(track_id):
    """
    Returns a list of selected information from Spotify's API given a song's URI

    Parameters:
        track_id (str): a track's URI
    
    Returns
        track (list): a list of song information including it's danceability, length, tempo, etc. 
    """
    md = sp.track(track_id)
    features = sp.audio_features(track_id)
    
    #meta data
    name = md['name']
    album = md['album']['name']
    artist = md['album']['artists'][0]['name']
    release_date = md['album']['release_date']
    length = md['duration_ms']
    popularity = md['popularity']
    
    #features from the data
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    key = features[0]['key']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    mode = features[0]['mode']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']
    valence = features[0]['valence']
    
    #putting it all together
    track = [name, album, artist, release_date, length, popularity, 
             acousticness, danceability, energy, instrumentalness, key,
            liveness, loudness, mode, speechiness, 
             tempo, time_signature, valence]
    return track

In [14]:
#Get track features for all tracks, append to a list
track_features = []
[track_features.append( getTrackFeatures(uri_df['uri'][i]) ) for i in range(1,len(uri_df['uri'])+1)]

print("All track features are accounted for!")

All track features are accounted for!


In [15]:
#Convert list to DataFrame
api_info_df = pd.DataFrame(track_features,columns=['name','album','artist',
    'release_date','length','popularity','acousticness','danceability',
    'energy','instrumentalness','key','liveness','loudness','mode',
    'speechiness','tempo','time_signature','valence'])

#change index to start at one
api_info_df.index = range(1,len(api_info_df)+1)

#change release_date to date format
api_info_df['release_date'] = pd.to_datetime(api_info_df['release_date'])

api_info_df

,name,album,artist,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
1,Closer,Closer,The Chainsmokers,2016-07-29,244960,84,0.41400,0.748,0.524,0.000000,8,0.1110,-5.599,1,0.0338,95.010,4,0.661
2,Fake Love,More Life,Drake,2017-03-18,210937,75,0.10500,0.928,0.481,0.000000,9,0.1760,-9.350,0,0.2870,134.007,4,0.613
3,Starboy,Starboy,The Weeknd,2016-11-25,230453,89,0.14100,0.679,0.587,0.000006,7,0.1370,-7.015,1,0.2760,186.003,4,0.486
4,Reminder,Starboy,The Weeknd,2016-11-25,218880,75,0.14200,0.706,0.505,0.000000,8,0.1640,-6.923,1,0.1930,160.053,4,0.388
5,Both (feat. Drake),The Return of East Atlanta Santa,Gucci Mane,2016-12-16,190311,68,0.11900,0.850,0.405,0.000118,7,0.0707,-7.509,0,0.2250,139.976,4,0.344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2239,Put It On Me,Realer 2,YoungBoy Never Broke Again,2022-09-07,152195,71,0.03310,0.665,0.666,0.000000,9,0.0841,-7.009,1,0.4290,164.205,4,0.844
2240,Shut Down,BORN PINK,BLACKPINK,2022-09-16,175889,86,0.00412,0.820,0.686,0.000000,0,0.1840,-5.102,1,0.0380,110.058,3,0.668
2241,Static,Gemini Rights,Steve Lacy,2022-07-15,156506,85,0.43400,0.335,0.305,0.631000,1,0.1010,-12.661,1,0.0741,79.001,5,0.215
2242,STAR WALKIN' (League of Legends Worlds Anthem),STAR WALKIN' (League of Legends Worlds Anthem),Lil Nas X,2022-09-22,210575,51,0.14300,0.634,0.691,0.000000,2,0.0970,-4.971,0,0.0458,141.831,4,0.316


## Joining all the data

Combine all the data collected/generated above! Leave the index as its own column so that each song index can be saved onto a Google Sheet.

In [16]:
#concatinate all variables into a final dataframe
final_df = pd.concat([gsheets_df.loc[:,:], uri_df.loc[:,'uri'], api_info_df.loc[:,'release_date':'valence'], uri_df.loc[:,'explicit']]
            ,axis=1)

#reset index and convert all integers/float/dates to strings in preparation to convert to a Google Shet
final_df = final_df.reset_index()

final_df

,index,title,artist,album,peak_date,streams_2017_to_present,total_streams_2020s,peak_week_streams,current_year,2022-09-16,2022-09-09,2022-09-02,2022-08-26,2022-08-19,2022-08-12,2022-08-05,2022-07-29,2022-07-22,2022-07-15,2022-07-08,2022-07-01,2022-06-24,2022-06-17,2022-06-10,2022-06-03,2022-05-27,2022-05-20,2022-05-13,2022-05-06,2022-04-29,2022-04-22,2022-04-15,2022-04-08,2022-04-01,2022-03-25,2022-03-18,2022-03-11,2022-03-04,2022-02-25,2022-02-18,2022-02-11,2022-02-04,2022-01-28,2022-01-21,2022-01-14,2022-01-07,2021-12-31,2021-12-24,2021-12-17,2021-12-10,2021-12-03,2021-11-26,2021-11-19,2021-11-12,2021-11-05,2021-10-29,2021-10-22,2021-10-15,2021-10-08,2021-10-01,2021-09-24,2021-09-17,2021-09-10,2021-09-03,2021-08-27,2021-08-20,2021-08-13,2021-08-06,2021-07-30,2021-07-23,2021-07-16,2021-07-09,2021-07-02,2021-06-25,2021-06-18,2021-06-11,2021-06-04,2021-05-28,2021-05-21,2021-05-14,2021-05-07,2021-04-30,2021-04-23,2021-04-16,2021-04-09,2021-04-02,2021-03-26,2021-03-19,2021-03-12,2021-03-05,2021-02-26,2021-02-19,2021-02-12,2021-02-05,2021-01-29,2021-01-22,2021-01-15,2021-01-08,2021-01-01,2020-12-25,2020-12-18,2020-12-11,2020-12-04,2020-11-27,2020-11-20,2020-11-13,2020-11-06,2020-10-30,2020-10-23,2020-10-16,2020-10-09,2020-10-02,2020-09-25,2020-09-18,2020-09-11,2020-09-04,2020-08-28,2020-08-21,2020-08-14,2020-08-07,2020-07-31,2020-07-24,2020-07-17,2020-07-10,2020-07-03,2020-06-26,2020-06-19,2020-06-12,2020-06-05,2020-05-29,2020-05-22,2020-05-15,2020-05-08,2020-05-01,2020-04-24,2020-04-17,2020-04-10,2020-04-03,2020-03-27,2020-03-20,2020-03-13,2020-03-06,2020-02-28,2020-02-21,2020-02-14,2020-02-07,2020-01-31,2020-01-24,2020-01-17,2020-01-10,2020-01-03,2019-12-27,2019-12-20,2019-12-13,2019-12-06,2019-11-29,2019-11-22,2019-11-15,2019-11-08,2019-11-01,2019-10-25,2019-10-18,2019-10-11,2019-10-04,2019-09-27,2019-09-20,2019-09-13,2019-09-06,2019-08-30,2019-08-23,2019-08-16,2019-08-09,2019-08-02,2019-07-26,2019-07-19,2019-07-12,2019-07-05,2019-06-28,2019-06-21,2019-06-14,2019-06-07,2019-05-31,2019-05-24,2019-05-17,2019-05-10,2019-05-03,2019-04-26,2019-04-19,2019-04-12,2019-04-05,2019-03-29,2019-03-22,2019-03-15,2019-03-08,2019-03-01,2019-02-22,2019-02-15,2019-02-08,2019-02-01,2019-01-25,2019-01-18,2019-01-11,2019-01-04,2018-12-28,2018-12-21,2018-12-14,2018-12-07,2018-11-30,2018-11-23,2018-11-16,2018-11-09,2018-11-02,2018-10-26,2018-10-19,2018-10-12,2018-10-05,2018-09-28,2018-09-21,2018-09-14,2018-09-07,2018-08-31,2018-08-24,2018-08-17,2018-08-10,2018-08-03,2018-07-27,2018-07-20,2018-07-13,2018-07-06,2018-06-29,2018-06-22,2018-06-15,2018-06-08,2018-06-01,2018-05-25,2018-05-18,2018-05-11,2018-05-04,2018-04-27,2018-04-20,2018-04-13,2018-04-06,2018-03-30,2018-03-23,2018-03-16,2018-03-09,2018-03-02,2018-02-23,2018-02-16,2018-02-09,2018-02-02,2018-01-26,2018-01-19,2018-01-12,2018-01-05,2017-12-29,2017-12-15,2017-12-08,2017-12-01,2017-11-24,2017-11-17,2017-11-10,2017-11-03,2017-10-27,2017-10-20,2017-10-13,2017-10-06,2017-09-29,2017-09-22,2017-09-15,2017-09-08,2017-09-01,2017-08-25,2017-08-18,2017-08-11,2017-08-04,2017-07-28,2017-07-21,2017-07-14,2017-07-07,2017-06-30,2017-06-23,2017-06-16,2017-06-09,2017-06-02,2017-05-26,2017-05-19,2017-05-12,2017-05-05,2017-04-28,2017-04-21,2017-04-14,2017-04-07,2017-03-31,total_2022,total_2021,total_2020,total_2019,total_2018,total_2017,units,holiday_music,remix_version,playlist_index,uri,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,explicit
0,1,Closer,The Chainsmokers,Closer,2016-09-09,10948414,0,5528407,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

## Converting the Dataframe to a Google Sheet

The codes above take a long time to run! To ease the data collection process and save time for research purposes, the dataframe is to be converted to a Google Sheet (same link with the HITS Streaming data) using the following code:

In [17]:
url = "https://docs.google.com/spreadsheets/d/165OdLYjLt4AgeqP5S5PunRonDkpp28nueHLFv994bPk/edit#gid=0"

spreadsheet_key = url.split("/")[-2]

wks_name = 'DF to Gspread'

scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('gspread_service_account.json', scope)

gc = gspread.authorize(credentials)

d2g.upload(final_df, spreadsheet_key, wks_name, credentials = credentials, row_names=False)

<Worksheet 'DF to Gspread' id:2071085388>

You can now use the worksheet above for analytical purposes!! Happy analyzing! 

# __Analysis on the data is in the second part of this project__
:)